In [11]:
%reset

In [12]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic, Matern, WhiteKernel, RBF
from sklearn.gaussian_process.kernels import Sum
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats

import numpy as np
import pandas as pd
import optuna
import warnings

In [13]:
OUTPUT_TEST = True

In [14]:
X_train = pd.read_csv("../data/cleaned/training.csv")
y_train = pd.read_csv("../data/cleaned/training_labels.csv")
X_val = pd.read_csv("../data/cleaned/validation.csv")
y_val = pd.read_csv("../data/cleaned/validation_labels.csv")

In [15]:
columns = X_train.columns
for col in columns:
    if '[' in col or ']' in col:
        old_name = col
        col = col.replace('[', '(')
        col = col.replace(']', ')')
        
        X_train = X_train.rename(columns={old_name:col})
        X_val = X_val.rename(columns={old_name:col})

In [16]:
X_train, X_verif, y_train, y_verif = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [17]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_verif = X_verif.reset_index(drop=True)
y_verif = y_verif.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [18]:
def define_kernel(trial):
    kernels = []
    n_kernels = trial.suggest_int('n_kernels', 1, 3)
    for i in range(n_kernels):
        kernel_type = trial.suggest_categorical(f'kernel_type_{i}', ["Matern", "RationalQuadratic", "WhiteKernel"])
        if kernel_type == 'RationalQuadratic':
            quad_params = {
                'length_scale': trial.suggest_float(f'RationalQuadratic_{i}_length_scale', 1e-5, 1e5),
                'alpha': trial.suggest_float(f'RationalQuadratic_{i}_alpha', 1e-5, 1e5)
            }   
            kernel = RationalQuadratic(length_scale=quad_params['length_scale'], alpha=quad_params['alpha'], length_scale_bounds=(1e-8,1e8))
        elif kernel_type == 'Matern':
            matern_params = {
                'length_scale': trial.suggest_float(f'Matern_{i}_length_scale', 1e-5, 1e5),
                'nu': trial.suggest_float(f'Matern_{i}_nu', 0.5, 5)
            }
            kernel = Matern(length_scale=matern_params['length_scale'], nu=matern_params['nu'], length_scale_bounds=(1e-8,1e8))
        elif kernel_type == "WhiteKernel":
            white_noise_params = {
                'noise_level': trial.suggest_float(f'WhiteKernel_{i}_noise_level', 1e-5, 1e5),
            }
            kernel = WhiteKernel(noise_level=white_noise_params['noise_level'])
        else:
            print("WRONG KERNEL NAME FOR:", kernel_type)
            TypeError
        kernels.append(kernel)

    if len(kernels) == 1:
        return kernels[0]
    else:
        combined_kernel = Sum(kernels[0], kernels[1])
        for j in range(1, n_kernels-1):
            combined_kernel = Sum(combined_kernel, kernels[j+1])
    return combined_kernel

In [19]:
def objective(trial):
    """Define the objective function"""
    kernel = define_kernel(trial)
    params = {
        'alpha': trial.suggest_float('alpha', 1e-3, 1e3, log=True),
        'n_restarts_optimizer': trial.suggest_int('n_restarts_optimizer', 0, 10),
    }

    params["kernel"] = kernel
    print(params)
    # Fit the model
    #bst = xgb.train(param, dtrain, evals=[(dtest, "validation")], callbacks=[pruning_callback])
    optuna_model = GaussianProcessRegressor(**params)
    #optuna_model.fit(X_train, y_train)
    
    batch_size = 500

    # Take a random sample of the DataFrame
    X_train_sampled = X_train.sample(n=batch_size)

    # Access the indexes of the sampled rows
    sampled_indexes = X_train_sampled.index
    y_train_sampled = y_train.loc[sampled_indexes]

    optuna_model.fit(X_train_sampled, y_train_sampled)
    
    # Make predictions
    verif_pred = optuna_model.predict(X_verif)
    train_pred = optuna_model.predict(X_train_sampled)
    verif_loss = mean_squared_error(y_verif,verif_pred,squared=False)
    train_loss = mean_squared_error(y_train_sampled,train_pred,squared=False)
    #val_loss_array.append(val_loss)
    #train_loss_array.append(train_loss)

    # Evaluate predictions
    error = abs(verif_loss - train_loss) + 2 * train_loss
    
    return error

In [20]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
study = optuna.create_study(pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=5)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-11-30 18:34:17,632] A new study created in memory with name: no-name-0d87d8c9-0ae4-497b-a1f1-88af119afc17


{'alpha': 0.08701801209740515, 'n_restarts_optimizer': 2, 'kernel': WhiteKernel(noise_level=7.92e+04) + Matern(length_scale=7.49e+04, nu=1.82)}


[I 2023-11-30 18:34:49,354] Trial 0 finished with value: 8.917633049120315 and parameters: {'n_kernels': 2, 'kernel_type_0': 'WhiteKernel', 'WhiteKernel_0_noise_level': 79162.75806060382, 'kernel_type_1': 'Matern', 'Matern_1_length_scale': 74867.98261295247, 'Matern_1_nu': 1.8180868883269663, 'alpha': 0.08701801209740515, 'n_restarts_optimizer': 2}. Best is trial 0 with value: 8.917633049120315.


{'alpha': 0.011002943458429866, 'n_restarts_optimizer': 4, 'kernel': WhiteKernel(noise_level=6.79e+04)}


[I 2023-11-30 18:34:49,948] Trial 1 finished with value: 38.33262499613891 and parameters: {'n_kernels': 1, 'kernel_type_0': 'WhiteKernel', 'WhiteKernel_0_noise_level': 67949.32089873607, 'alpha': 0.011002943458429866, 'n_restarts_optimizer': 4}. Best is trial 0 with value: 8.917633049120315.


...

{'alpha': 41.163692461828404, 'n_restarts_optimizer': 10, 'kernel': WhiteKernel(noise_level=9.98e+04) + WhiteKernel(noise_level=2.02e+04) + RationalQuadratic(alpha=6.69e+04, length_scale=2.22e+04)}


c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__alpha is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\nickr\anaconda3\envs\ml_project\

{'alpha': 1.5906844986177993, 'n_restarts_optimizer': 6, 'kernel': WhiteKernel(noise_level=9.09e+04)}


[I 2023-11-30 18:35:31,439] Trial 4 finished with value: 38.09848837314606 and parameters: {'n_kernels': 1, 'kernel_type_0': 'WhiteKernel', 'WhiteKernel_0_noise_level': 90890.843576057, 'alpha': 1.5906844986177993, 'n_restarts_optimizer': 6}. Best is trial 0 with value: 8.917633049120315.


Best trial:
  Value:  8.917633049120315
  Params: 
    n_kernels: 2
    kernel_type_0: WhiteKernel
    WhiteKernel_0_noise_level: 79162.75806060382
    kernel_type_1: Matern
    Matern_1_length_scale: 74867.98261295247
    Matern_1_nu: 1.8180868883269663
    alpha: 0.08701801209740515
    n_restarts_optimizer: 2


In [21]:
def reconstruct_kernel(encoding):
    n_kernels = encoding['n_kernels']
    kernels = []
    for i in range(n_kernels):
        kernel_type = encoding[f'kernel_type_{i}']
        length_scale = encoding.get(f'{kernel_type}_{i}_length_scale', None)
        nu = encoding.get(f'{kernel_type}_{i}_nu', None)

        if kernel_type == 'Matern':
            kernel = Matern(length_scale=length_scale, nu=nu)
        elif kernel_type == 'RationalQuadratic':
            alpha = encoding.get(f'{kernel_type}_{i}_alpha', 1.0)
            kernel = RationalQuadratic(length_scale=length_scale, alpha=alpha)
        elif kernel_type == 'WhiteKernel':
            noise_level = encoding.get(f'{kernel_type}_{i}_noise_level', 1.0)
            print(noise_level)
            kernel = WhiteKernel(noise_level=noise_level)
        # Add more conditions for other kernel types if needed

        kernels.append(kernel)

    # Sum the individual kernels to get the final composite kernel
    if len(kernels) == 1:
        return kernels[0]
    else:
        final_kernel = Sum(kernels[0], kernels[1])
        for j in range(1, n_kernels-1):
            final_kernel = Sum(final_kernel, kernels[j+1])

    return final_kernel

In [22]:
#Check performance with no tuning to ensure performance is improving
sanity_check = GaussianProcessRegressor(kernel=RBF())
sanity_check.fit(X_train, y_train)
val_pred = sanity_check.predict(X_val)
verif_pred = sanity_check.predict(X_verif)
sanity_verif_error = mean_squared_error(y_verif,verif_pred,squared=False)
sanity_val_error = mean_squared_error(y_val,val_pred,squared=False)
print("SANITY CHECK VALUES:")
print("Verification RMSE:", sanity_verif_error)
print("Validation RMSE:", sanity_val_error)

c:\Users\nickr\anaconda3\envs\ml_project\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


SANITY CHECK VALUES:
Verification RMSE: 19.204835218127272
Talidation RMSE: 19.094270223829994


In [23]:
params = trial.params
print(params)
kernel = reconstruct_kernel(params)
print(kernel)
gp = GaussianProcessRegressor(kernel=kernel,alpha=params['alpha'], n_restarts_optimizer=params['n_restarts_optimizer'])

{'n_kernels': 2, 'kernel_type_0': 'WhiteKernel', 'WhiteKernel_0_noise_level': 79162.75806060382, 'kernel_type_1': 'Matern', 'Matern_1_length_scale': 74867.98261295247, 'Matern_1_nu': 1.8180868883269663, 'alpha': 0.08701801209740515, 'n_restarts_optimizer': 2}
79162.75806060382
WhiteKernel(noise_level=7.92e+04) + Matern(length_scale=7.49e+04, nu=1.82)


In [24]:
gp.fit(X_train, y_train)

In [ ]:
val_pred, std_prediction = gp.predict(X_val, return_std=True)
error = mean_squared_error(y_val,val_pred,squared=False)
print("RMSE:", error)
print("Difference from sanity check:", sanity_val_error - error)

In [ ]:
if not OUTPUT_TEST:
    raise ValueError("OUTPUT_TEST set to False. If you would like to output final test values set to True and continue running from here")

In [ ]:
X_test = pd.read_csv("../data/cleaned/test.csv")
y_test = pd.read_csv("../data/cleaned/test_labels.csv")

In [ ]:
columns = X_test.columns
for col in columns:
    if '[' in col or ']' in col:
        old_name = col
        col = col.replace('[', '(')
        col = col.replace(']', ')')
        
        X_test = X_test.rename(columns={old_name:col})

In [ ]:
test_preds = gp.predict(X_test)
train_preds = gp.predict(X_train)

In [ ]:
#Save test true vals and predictions to csv

pred_data = pd.DataFrame(test_preds)
pred_filepath = '../data/predictions/GP/test_pred_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
pred_data = pd.DataFrame(y_test)
pred_filepath = '../data/predictions/GP/test_true_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)

#Save train true vals and predictions to csv

pred_data = pd.DataFrame(train_preds)
pred_filepath = '../data/predictions/GP/train_pred_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
pred_data = pd.DataFrame(y_train)
pred_filepath = '../data/predictions/GP/train_true_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)

In [ ]:
#Save inputs to csv

pred_data = pd.DataFrame(X_train)
pred_filepath = '../data/predictions/GP/train_input_gp.csv'
pred_data.to_csv(pred_filepath, index=False, header=False)
true_data = pd.DataFrame(X_test)
true_filepath = '../data/predictions/GP/test_input_gp.csv'
true_data.to_csv(true_filepath, index=False, header=False)

In [ ]:
#Read in values from csv and calculate RMSE and r values

test_pred_data = np.genfromtxt('../data/predictions/GP/test_pred_gp.csv', delimiter=',', filling_values=np.nan)
test_true_data = np.genfromtxt('../data/predictions/GP/test_true_gp.csv', delimiter=',', filling_values=np.nan)
train_pred_data = np.genfromtxt('../data/predictions/GP/train_pred_gp.csv', delimiter=',', filling_values=np.nan)
train_true_data = np.genfromtxt('../data/predictions/GP/train_true_gp.csv', delimiter=',', filling_values=np.nan)

test_rmse = mean_squared_error(test_true_data,test_pred_data,squared=False)
test_r = stats.pearsonr(test_true_data,test_pred_data)

train_rmse = mean_squared_error(train_true_data,train_pred_data,squared=False)
train_r = stats.pearsonr(train_true_data,train_pred_data)

print("Train:")
print(train_rmse)
print('Test:')
print(test_rmse)
print(test_r)